In [1]:
import cv2
from IPython.display import Image, display
import numpy as np
import glob
import pandas as pd

def imshow(img):
    ret, encoded = cv2.imencode(".jpg", img)
    display(Image(encoded))

# Get image path
df = pd.read_csv("Image_Size.csv")

In [2]:
df

,ID,height,width
0,1_20130812_132022_l_カラー眼底.png,1312,2000
1,1_20130812_132022_r_カラー眼底.png,1312,2000
2,2_20130225_104034_r_カラー眼底.png,1024,1360
3,2_20130225_104034_l_カラー眼底.png,1024,1360
4,3_20180417_115422_r_カラー眼底.png,2448,3696
...,...,...,...
683,c100_20200925_151036_l_カラー眼底.png,2448,3696
684,c101_20210208_161522_l_カラー眼底.png,2448,3696
685,c102_20220118_111409_r_カラー眼底.png,2448,3696
686,c103_20211116_120920_r_カラー眼底.png,2448,3696


In [3]:
df["cx"] = 0
df["cy"] = 0
df["r"] = 0

# height = 2448
df.loc[df["height"]==2448,"cx"] = 1820
df.loc[df["height"]==2448,"cy"] = 1180
df.loc[df["height"]==2448,"r"] = 1316

#height = 1312
df.loc[df["height"]==1312,"cx"] = 1006
df.loc[df["height"]==1312,"cy"] = 624
df.loc[df["height"]==1312,"r"] = 837

# height = 1024
df.loc[df["height"]==1024,"cx"] = 701
df.loc[df["height"]==1024,"cy"] = 505
df.loc[df["height"]==1024,"r"] = 553

# height = 872 
df.loc[df["height"]==872,"cx"] = 601
df.loc[df["height"]==872,"cy"] = 453
df.loc[df["height"]==872,"r"] = 560

In [4]:
df

,ID,height,width,cx,cy,r
0,1_20130812_132022_l_カラー眼底.png,1312,2000,1006,624,837
1,1_20130812_132022_r_カラー眼底.png,1312,2000,1006,624,837
2,2_20130225_104034_r_カラー眼底.png,1024,1360,701,505,553
3,2_20130225_104034_l_カラー眼底.png,1024,1360,701,505,553
4,3_20180417_115422_r_カラー眼底.png,2448,3696,1820,1180,1316
...,...,...,...,...,...,...
683,c100_20200925_151036_l_カラー眼底.png,2448,3696,1820,1180,1316
684,c101_20210208_161522_l_カラー眼底.png,2448,3696,1820,1180,1316
685,c102_20220118_111409_r_カラー眼底.png,2448,3696,1820,1180,1316
686,c103_20211116_120920_r_カラー眼底.png,2448,3696,1820,1180,1316


In [5]:
unique_combinations = df.groupby(['height', 'width', 'cx', 'cy', 'r']).size().reset_index().rename(columns={0: 'count'})

print(unique_combinations)

   height  width    cx    cy     r  count
0     872   1200   601   453   560      2
1    1024   1360   701   505   553    145
2    1312   2000  1006   624   837    279
3    2448   3696  1820  1180  1316    262


In [6]:
df.to_csv("circle_detection_size.csv",index=False)

In [7]:
df

,ID,height,width,cx,cy,r
0,1_20130812_132022_l_カラー眼底.png,1312,2000,1006,624,837
1,1_20130812_132022_r_カラー眼底.png,1312,2000,1006,624,837
2,2_20130225_104034_r_カラー眼底.png,1024,1360,701,505,553
3,2_20130225_104034_l_カラー眼底.png,1024,1360,701,505,553
4,3_20180417_115422_r_カラー眼底.png,2448,3696,1820,1180,1316
...,...,...,...,...,...,...
683,c100_20200925_151036_l_カラー眼底.png,2448,3696,1820,1180,1316
684,c101_20210208_161522_l_カラー眼底.png,2448,3696,1820,1180,1316
685,c102_20220118_111409_r_カラー眼底.png,2448,3696,1820,1180,1316
686,c103_20211116_120920_r_カラー眼底.png,2448,3696,1820,1180,1316


In [8]:
i = 0
print(i)
picture_path = df.iat[i,0]
path = "../../storage/data/" + picture_path[0:-3] + "jpg"
# Load data
img = cv2.imread(path)
#cx, cy, r = circles[0][0]
cx, cy, r = df.iat[i,3],df.iat[i,4],df.iat[i,5]

# Caliculate Circle
target_height = r*2

height,width = img.shape[:2]

target_size = (target_height,width) #src size < dst sizeの前提

top = int(r-cy)
bottom = int(target_size[0] - height - top)
left = int((target_size[1] - width)/2)
right = int(target_size[1] - width - left)

color = np.nan #(0,0,0)
img_filled = cv2.copyMakeBorder(img, top, bottom, left, right,cv2.BORDER_CONSTANT,value=color)
cx = int(cx)
cy = int(cy)
r = int(r)
# Write Circle
cv2.circle(img_filled, (cx, r), r, (0, 255, 0), 2)
# Write Center
cv2.circle(img_filled, (cx, r), 2, (0, 255, 0), 2)

mask = np.full(img_filled.shape[:2],0,dtype = img_filled.dtype)
cv2.circle(mask, (cx, r), r,color = 255, thickness = -1)
img_nan = np.full(img_filled.shape,np.nan,dtype = img_filled.dtype)
image_processed = img_filled
image_processed[mask == 0] = img_nan[mask == 0]
file_png = '../circle_detection/images/'+ picture_path
cv2.imwrite(file_png,image_processed)

0


False

In [9]:
for i in range(len(df)):
    print(i)
    picture_path = df.iat[i,0]
    path = "../../storage/data/" + picture_path[0:-3] + "jpg"
    # Load data
    img = cv2.imread(path)
    #cx, cy, r = circles[0][0]
    cx, cy, r = df.iat[i,3],df.iat[i,4],df.iat[i,5]

    # Caliculate Circle
    target_height = r*2

    height,width = img.shape[:2]

    target_size = (target_height,width) #src size < dst sizeの前提

    top = int(r-cy)
    bottom = int(target_size[0] - height - top)
    left = int((target_size[1] - width)/2)
    right = int(target_size[1] - width - left)

    color = np.nan #(0,0,0)
    img_filled = cv2.copyMakeBorder(img, top, bottom, left, right,cv2.BORDER_CONSTANT,value=color)
    cx = int(cx)
    cy = int(cy)
    r = int(r)
    # Write Circle
    cv2.circle(img_filled, (cx, r), r, (0, 255, 0), 2)
    # Write Center
    cv2.circle(img_filled, (cx, r), 2, (0, 255, 0), 2)

    mask = np.full(img_filled.shape[:2],0,dtype = img_filled.dtype)
    cv2.circle(mask, (cx, r), r,color = 255, thickness = -1)
    img_nan = np.full(img_filled.shape,np.nan,dtype = img_filled.dtype)
    image_processed = img_filled
    image_processed[mask == 0] = img_nan[mask == 0]
    file_png = '../circle_detection/images/'+ picture_path
    cv2.imwrite(file_png,image_processed)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27